In [144]:
import requests

API_KEY = 'dce8f039-d69a-4912-ac86-9ddba0f874a7'  # Substitua pela sua chave de API real
BASE_URL = 'https://api.moskitcrm.com/v2/products'

In [145]:
def get_all_activities(credencials_keys, api_url ,order=None, quantity=10, sort=None, start = 0, max_pages=11):
    all_data = []
    next_page_token = None
    pages_retrieved = 0

    for _ in range(max_pages):
        result = get_activities(credencials_keys, api_url, order=order, quantity=quantity, sort=sort, start=start, nextPageToken=next_page_token)
        
        if 'error' in result:
            print("Erro:", result['error'])
            break
        
        all_data.extend(result['data'])
        next_page_token = result.get('nextPageToken')

        # Se não houver mais tokens para a próxima página, saímos do loop
        if not next_page_token:
            break
        
        # Atualiza o valor de start para refletir o início da próxima página
        start += quantity
        pages_retrieved += 1

    if pages_retrieved >= max_pages:
        print("Atingido o número máximo de páginas. Pode haver mais dados disponíveis.")

    return all_data

In [146]:
def get_activities(credencials_keys, api_url, order=None, quantity=10, sort=None, start=0, nextPageToken=None):
    headers = {
        'apikey': credencials_keys
    }

    params = {
        'order': order,
        'quantity': quantity,
        'sort': sort,
        'start': start
    }

    if nextPageToken:
        params['nextPageToken'] = nextPageToken

    try:
        response = requests.get(api_url, headers=headers, params=params)
        response.raise_for_status()

        # Pega o token da próxima página dos headers da resposta
        next_page_token = response.headers.get('X-Moskit-Listing-Next-Page-Token', None)
        
        return {
            'data': response.json(),
            'nextPageToken': next_page_token
        }
    except requests.exceptions.RequestException as e:
        return {'error': str(e)}

In [147]:
def get_total_records():
    headers = {
        'apikey': API_KEY
    }

    try:
        # Faz uma requisição ao endpoint
        response = requests.get(BASE_URL, headers=headers)
        response.raise_for_status()

        # Pega o total de registros dos headers da resposta
        total_records = response.headers.get('X-Moskit-Listing-Total', 'Não disponível')
        
        return total_records
    except requests.exceptions.RequestException as e:
        return {'error': str(e)}

In [148]:
if __name__ == '__main__':
    # Obter o total de registros
    total_records = get_total_records()
    
    # Garantir que total_records é um número válido
    try:
        total_records = int(total_records)
    except ValueError:
        print(f"Erro ao converter total_records para int: {total_records}")
        total_records = 0

    all_activities = []
    start = 0
    quantity = 50
    max_pages_per_loop = int(total_records/quantity) + 2
    
    while len(all_activities) < total_records:
        # Obter atividades em um loop
        activities = get_all_activities(API_KEY, BASE_URL ,order='asc', quantity=quantity, sort='id', start=start, max_pages=max_pages_per_loop)
        if not activities:
            break

        all_activities.extend(activities)
        # start = len(all_activities)

        if len(all_activities) >= total_records:
            break

    # Extrair a lista de IDs
    list_id = [activity['id'] for activity in all_activities]
    
    # Contar o número de IDs únicos
    unique_count = len(set(list_id))
    
    print(f"Número de registros obtidos: {len(all_activities)}")
    print(f"Número de IDs únicos: {unique_count}")

Número de registros obtidos: 4747
Número de IDs únicos: 4747


In [149]:
total_records

4747